In [1]:
import datetime
import gc
import math
import os

import lightgbm as lgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from tqdm.notebook import tqdm

In [2]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', 20)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

## 拼接文件

In [3]:
new_train_path = '../../input/train_time'
new_test_path = '../../input/test_time'

new_train_files = os.listdir(new_train_path)
new_test_files = os.listdir(new_test_path)

In [4]:
ret = []
for file in tqdm(new_train_files):
    df_train = pd.read_csv(f'{new_train_path}/{file}')
    df_train = df_train.drop_duplicates(['x', 'y'], keep='last')
    ret.append(df_train) 
df_train = pd.concat(ret)
df_train.to_hdf('../../input/train_id_all.h5', 'df', mode='w')

KeyboardInterrupt: 

In [5]:
del ret
gc.collect()

64

In [6]:
ret = []
for file in tqdm(new_test_files):
    df_test = pd.read_csv(f'{new_test_path}/{file}')
    ret.append(df_test)
df_test = pd.concat(ret)
df_test.to_hdf('../../input/test_id_all.h5', 'df', mode='w')

In [7]:
del ret
gc.collect()

64

## 读取文件

In [30]:
# 读取文件
df_train = pd.read_hdf('../../input/train_id_all.h5')
df_test = pd.read_hdf('../../input/test_id_all.h5')

In [31]:
df_train.shape

(1173557, 120)

In [32]:
df_test.shape

(782378, 119)

In [33]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_test = df_test.drop(['Unnamed: 0'], axis=1)

In [34]:
df_train.head(3)

,渔船ID,x,y,速度,方向,time,type,sin_angle,cos_angle,month,day,hour,minute,second,time_past_sec,time_step_sec,x_past,y_past,x_step,y_step,xy_distance_step,xy_velocity_step,sin_hour,cos_hour,sin_minute,cos_minute,sin_second,cos_second,x_max,x_min,x_mean,x_median,x_std,x_skew,x_mode,y_max,y_min,y_mean,y_median,y_std,y_skew,y_mode,day_max,day_min,day_mean,day_median,day_std,day_skew,day_mode,速度_max,速度_min,速度_mean,速度_median,速度_std,速度_skew,速度_mode,xy_velocity_step_max,xy_velocity_step_min,xy_velocity_step_mean,xy_velocity_step_median,xy_velocity_step_std,xy_velocity_step_skew,xy_velocity_step_mode,xy_distance_step_max,xy_distance_step_min,xy_distance_step_mean,xy_distance_step_median,xy_distance_step_std,xy_distance_step_skew,xy_distance_step_mode,time_step_sec_max,time_step_sec_min,time_step_sec_mean,time_step_sec_median,time_step_sec_std,time_step_sec_skew,time_step_sec_mode,x_step_max,x_step_min,x_step_mean,x_step_median,x_step_std,x_step_skew,x_step_mode,y_step_max,y_step_min,y_step_mean,y_step_median,y_step_std,y_step_skew,y_step_mode,sin_angle_max,sin_angle_min,sin_angle_mean,sin_angle_median,sin_angle_std,sin_angle_skew,sin_angle_mode,cos_angle_max,cos_angle_min,cos_angle_mean,cos_angle_median,cos_angle_std,cos_angle_skew,cos_angle_mode,day_past,x_past_max,y_past_max,square_area,long_stop_1000_count,long_stop_600_count,long_stop_1000_rate,long_stop_600_rate,short_report_count,short_report_rete,no_move_distance_count,no_move_distance_rate,daytime_count,daytime_rate
0,0,6.152038e+06,5.124873e+06,2.59,102,1900-11-10 11:58:19,拖网,0.978148,-0.207912,11,10,11,58,19,85731,600,33686.667453,-5798.943044,808.914660,-344.632139,879.269150,1.463010,0.258819,-0.965926,-0.207912,0.978148,0.913545,-0.406737,6.152038e+06,6.118352e+06,6.119351e+06,6.118352e+06,5037.320747,5.255558,6.118352e+06,5.130781e+06,5.124873e+06,5.130494e+06,5.130672e+06,850.264541,-4.762308,5.130672e+06,10,7,8.502415,8.5,0.950975,0.001538,8.0,9.39,0.0,0.265966,0.0,1.321248,5.520205,0.0,5.115562,0.0,0.136127,0.0,0.67907,5.515274,0.0,4745.887438,0.0,86.879282,0.0,449.486694,6.277462,0.0,2343,0,624.471014,600.0,171.039545,4.491647,600.0,3816.791567,-1.939802,81.368762,0.0,420.81168,5.88769,0.0,620.168819,-2820.558404,-14.007109,0.0,160.276396,-13.645926,0.0,1.0,0.0,0.043957,0.0,0.201004,4.378414,0.0,1.0,-0.62932,0.945962,1.0,0.25114,-4.607238,1.0,3,33686.667453,5907.975523,1.990200e+08,18,80,0.043478,0.193237,2,0.004831,391,0.944444,228,0.550725
1,0,6.151230e+06,5.125218e+06,2.70,113,1900-11-10 11:48:19,拖网,0.920505,-0.390731,11,10,11,48,19,85131,600,32877.752793,-5454.310904,808.872353,-344.699932,879.256804,1.462990,0.258819,-0.965926,-0.951057,0.309017,0.913545,-0.406737,6.152038e+06,6.118352e+06,6.119351e+06,6.118352e+06,5037.320747,5.255558,NaN,5.130781e+06,5.124873e+06,5.130494e+06,5.130672e+06,850.264541,-4.762308,NaN,10,7,8.502415,8.5,0.950975,0.001538,NaN,9.39,0.0,0.265966,0.0,1.321248,5.520205,NaN,5.115562,0.0,0.136127,0.0,0.67907,5.515274,NaN,4745.887438,0.0,86.879282,0.0,449.486694,6.277462,NaN,2343,0,624.471014,600.0,171.039545,4.491647,NaN,3816.791567,-1.939802,81.368762,0.0,420.81168,5.88769,NaN,620.168819,-2820.558404,-14.007109,0.0,160.276396,-13.645926,NaN,1.0,0.0,0.043957,0.0,0.201004,4.378414,NaN,1.0,-0.62932,0.945962,1.0,0.25114,-4.607238,NaN,3,33686.667453,5907.975523,1.990200e+08,18,80,0.043478,0.193237,2,0.004831,391,0.944444,228,0.550725
2,0,6.150421e+06,5.125563e+06,2.70,116,1900-11-10 11:38:19,拖网,0.898794,-0.438371,11,10,11,38,19,84531,600,32068.880440,-5109.610973,808.830024,-344.767718,879.244441,1.462969,0.258819,-0.965926,-0.743145,-0.669131,0.913545,-0.406737,6.152038e+06,6.118352e+06,6.119351e+06,6.118352e+06,5037.320747,5.255558,NaN,5.130781e+06,5.124873e+06,5.130494e+06,5.130672e+06,850.264541,-4.762308,NaN,10,7,8.502415,8.5,0.950975,0.001538,NaN,9.39,0.0,0.265966,0.0,1.321248,5.520205,NaN,5.115562,0.0,0.136127,0.0,0.67907,5.515274,NaN,4745.887438,0.0,86.879282,0.0,449.486694,6.277462,NaN,2343,0,624.471014,600.0,171.039545,4.491647,NaN,3816.791

In [35]:
def drop_unnecessary_col(df):
    unneccessary = [# 'x', 'y', 'time', '速度', 'time',  # 结束时候的原始数据
                    'time',
                    'sin_angle', 'cos_angle',
                    'month', 'day', 'hour', 'minute', 'second', 'time_past_sec', 'time_step_sec',
                    'x_past', 'y_past', 'x_step', 'y_step', 
                    'xy_distance_step', 'xy_velocity_step', 
                    'sin_hour', 'cos_hour', 
                    'sin_minute', 'cos_minute',
                    'sin_second', 'cos_second',
                    # 'x_max', 'x_min', 'x_mean', 'x_median', 'x_std', 'x_skew', 'x_mode',
                    # 'y_max', 'y_min', 'y_mean', 'y_median', 'y_std', 'y_skew', 'y_mode', 
                    'day_max', 'day_min', 'day_mean', 'day_median', 'day_std', 'day_skew', 'day_mode',
                    #'速度_max', '速度_min', '速度_mean', '速度_median', '速度_std', '速度_skew', '速度_mode',
                    'xy_velocity_step_max', 'xy_velocity_step_min', 
                    'xy_velocity_step_mean', 
                    'xy_velocity_step_median', 'xy_velocity_step_std', 'xy_velocity_step_skew', 'xy_velocity_step_mode',
                    'xy_distance_step_max', 'xy_distance_step_min', 
                    'xy_distance_step_mean', 
                    'xy_distance_step_median', 'xy_distance_step_std', 'xy_distance_step_skew', 'xy_distance_step_mode',
                    'time_step_sec_max', 
                    'time_step_sec_min', 
                    'time_step_sec_mean', 
                    'time_step_sec_median', 'time_step_sec_std', 'time_step_sec_skew', 'time_step_sec_mode',
                    'x_step_max', 
                    'x_step_min', 
                    'x_step_mean', 'x_step_median', 'x_step_std', 'x_step_skew', 'x_step_mode', 
                    'y_step_max', 
                    'y_step_min', 
                    'y_step_mean', 'y_step_median', 'y_step_std', 'y_step_skew', 'y_step_mode', 
                    'sin_angle_max','sin_angle_min', 'sin_angle_mean', 'sin_angle_median', 'sin_angle_std', 'sin_angle_skew', 'sin_angle_mode',
                    'cos_angle_max','cos_angle_min', 'cos_angle_mean', 'cos_angle_median', 'cos_angle_std', 'cos_angle_skew', 'cos_angle_mode',
                    'day_past',
                    'x_past_max', 'y_past_max', 
                    'square_area', 
                    'long_stop_1000_count', 
                    'long_stop_1000_rate', 
                    'long_stop_600_count', 
                    'long_stop_600_rate',
                    'short_report_count', 'short_report_rete',
                    'no_move_distance_count', 
                    'no_move_distance_rate',
                    'daytime_count', 
                    'daytime_rate',
                    ]
    df = df.drop(unneccessary, axis=1)
    
    return df

In [36]:
df_train_clean = drop_unnecessary_col(df_train.copy())
df_test_clean = drop_unnecessary_col(df_test.copy())

In [37]:
df_train_clean.head(3)

,渔船ID,x,y,速度,方向,type,x_max,x_min,x_mean,x_median,x_std,x_skew,x_mode,y_max,y_min,y_mean,y_median,y_std,y_skew,y_mode,速度_max,速度_min,速度_mean,速度_median,速度_std,速度_skew,速度_mode
0,0,6.152038e+06,5.124873e+06,2.59,102,拖网,6.152038e+06,6.118352e+06,6.119351e+06,6.118352e+06,5037.320747,5.255558,6.118352e+06,5.130781e+06,5.124873e+06,5.130494e+06,5.130672e+06,850.264541,-4.762308,5.130672e+06,9.39,0.0,0.265966,0.0,1.321248,5.520205,0.0
1,0,6.151230e+06,5.125218e+06,2.70,113,拖网,6.152038e+06,6.118352e+06,6.119351e+06,6.118352e+06,5037.320747,5.255558,NaN,5.130781e+06,5.124873e+06,5.130494e+06,5.130672e+06,850.264541,-4.762308,NaN,9.39,0.0,0.265966,0.0,1.321248,5.520205,NaN
2,0,6.150421e+06,5.125563e+06,2.70,116,拖网,6.152038e+06,6.118352e+06,6.119351e+06,6.118352e+06,5037.320747,5.255558,NaN,5.130781e+06,5.124873e+06,5.130494e+06,5.130672e+06,850.264541,-4.762308,NaN,9.39,0.0,0.265966,0.0,1.321248,5.520205,NaN


In [38]:
df_train_clean.shape

(1173557, 27)

### 训练

In [39]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn import preprocessing

In [40]:
label_dict = {"拖网": 0,
             "围网": 1,
             "刺网": 2}

df_train_clean['type'] = df_train_clean['type'].apply(lambda x: label_dict[x])

In [41]:
df_train_clean = df_train_clean.drop_duplicates(['x', 'y'], keep='last')

In [42]:
df_train_clean.shape

(1064396, 27)

In [45]:
X = df_train_clean.drop(['type', '渔船ID'], axis=1)
y = df_train_clean['type']

In [46]:
clf = xgb.XGBClassifier(max_depth=6, 
                          learning_rate=0.01, 
                          n_estimators=1000, 
                          objective='multi:softmax',
                          n_jobs=6, 
                          )


fold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
models = []

for index, (train_idx, val_idx) in enumerate(fold.split(X, y)):
    clf.fit(X.iloc[train_idx], y.iloc[train_idx])
    # val_pred = model.predict_proba(X.iloc[val_idx])
    val_pred = clf.predict(X.iloc[val_idx])
    val_y = y.iloc[val_idx]
    print(index, 'val f1', metrics.f1_score(val_y, val_pred, average='macro'))

0 val f1 0.9877384948856415
1 val f1 0.9878123349758522
2 val f1 0.9894989791908686


In [50]:
submit = clf.predict(df_test_clean.drop(['渔船ID'], axis=1))

In [51]:
df_test_clean['type'] = submit

In [66]:
result = df_test_clean.groupby('渔船ID', as_index=False)['type'].agg(pd.Series.mode)

In [67]:
result['type'].value_counts()

0    1207
1     475
2     318
Name: type, dtype: int64

In [68]:
result

,渔船ID,type
0,7000,1
1,7001,0
2,7002,1
3,7003,0
4,7004,1
...,...,...
1995,8995,1
1996,8996,1
1997,8997,2
1998,8998,0


In [69]:
label_dict_rev = {0: "拖网",
                  1: "围网",
                  2: "刺网"}

result['type'] = result['type'].apply(lambda x: label_dict_rev[x])

In [70]:
result

,渔船ID,type
0,7000,围网
1,7001,拖网
2,7002,围网
3,7003,拖网
4,7004,围网
...,...,...
1995,8995,围网
1996,8996,围网
1997,8997,刺网
1998,8998,拖网


In [72]:
result.to_csv("sub_0117.csv", encoding='utf-8')